In [131]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import preprocessing
from sklearn import tree
from typing import List
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [132]:
import random
import math
def generateData(n):
    data = []
    for i in range(0,n):
        minggu = random.randint(1, 520)
        totaldonor = 1 if math.floor(minggu/8) == 0 else random.randint(1,math.floor(minggu/8))
        data.append([random.randint(1, 520), totaldonor])
    return data


In [133]:

data = [
    [200,1],
    [8,1],
    [4,1],
    [16,4],
    [308,24],
    [4,4],
    [200,2],
    [200,14],
    [13,1],
    [200,20],
    [28,2],
    [730,75],
    [16,2],
    [26,3],
    [26,2],
    [520,44],
    [52,3],
    [128,3],
    [15,2],
    [52,4],
    [10,0],
    [8,0],
    [52,5],
    [3,1],
    [4,2],
    [92,4],
    [52,4],
    [90,3],
    [3,1],
    [1,1],
    [112,10],
    [52,0],
]
# print(generateData(100))
data = generateData(300)
dataset_MLP = []
dataset_DT = []
for i,d in enumerate(data):
    minggu = d[0]
    totaldonor = d[1]
    # todo: buat pandasnya dengan nama kolom masing-masing
    dataset_MLP.append([minggu, totaldonor,totaldonor/minggu]) # menghitung "seberapa sering" valuenya 0-1
    dataset_DT.append([(minggu/totaldonor) if totaldonor != 0 else 0,totaldonor/minggu, 0.0 if totaldonor == 0 else 1.0 if minggu/totaldonor <= 12.0 else 0.0])
dataset_MLP = pd.DataFrame(dataset_MLP,columns=["Total Minggu","Total donor darah","frekuensi"])
dataset_DT = pd.DataFrame(dataset_DT,columns=["Rata-rata Interval donor","Frekuensi","Aktif"])

# real_df = pd.DataFrame(data,columns=["Jumlah Minggu", "Total donor per tahun","frekuensi"])
# real_df.head()
from IPython.display import display, HTML
# tranfusion_df
display(HTML("<h2>Dataset Rekayasa untuk training Deep Learning (MLP) dan Decision Tree (DT)</h2>"))
display(HTML(f"<div style='display:flex;justify-content:space-between;overflow-y:auto;max-height:300px'><div><h2>Dataset MLP</h2>{dataset_MLP.to_html()}</div><div><h2>Dataset DT</h2>{dataset_DT.to_html()}</div></div>"))

,Total Minggu,Total donor darah,frekuensi
0,247,3,0.012146
1,106,22,0.207547
2,302,7,0.023179
3,53,1,0.018868
4,118,34,0.288136
5,96,4,0.041667
6,478,28,0.058577
7,380,17,0.044737
8,292,7,0.023973
9,483,1,0.002070


In [134]:
scalerX = MinMaxScaler()


def preprocessing(df:pd.DataFrame,categorical_columns:List[str],scaling:bool, unused_columns:List[str], label:str):
  df.dropna() # drop row bila salah 1 atau lebih ada NaN
  # df.dropna(subset=[1]) # drop row bila column 1 NaN

  if len(categorical_columns) > 0:
    df = pd.get_dummies(data=df,columns=categorical_columns)
  # for column in df.columns:
  #   le = LabelEncoder()
  #   le.fit(df[column])
  #   df[column] = le.transform(df[column])
  
  # harus displit dulu antara features dan label, karena bila di minmax scalling dulu
  # nama kolom akan hilang, berubah menjadi array 2d
  unused_columns.append(label) # hapus kolom data column dan juga label
  X = df.drop(columns=unused_columns) 
  Y = df[label]
  Y = np.array(Y).reshape((-1,1))
  if scaling:
    global scalerX
    scalerX.fit(X.to_numpy())
    X = scalerX.transform(X.to_numpy())

    # scaler = MinMaxScaler()
    # scaler.fit(Y)
    # Y = scaler.transform(Y)

  # always produce the same result (only random once)
  # x_train, x_test = train_test_split(X, test_size=test_size, random_state=25)
  # y_train, y_test = train_test_split(Y, test_size=test_size, random_state=25)

 # convert to np arrays
  X = np.array(X)
  Y = np.array(Y).reshape((-1,1))
  
  return X, Y

# def scale(X):
#     global scalerX
#     # Xzero = scaler0.transform([X[0]])
#     # Xone = scaler1.transform([X[1]])
#     df = pd.DataFrame([X],columns=["Donor 1 tahun terakhir", "frekuensi donor per tahun"])

#     print("DF")
#     print(df)
#     # result = scalerX.transform(df)
#     result = scalerX.transform([X])
#     return result

In [135]:
scaled_x_train, scaled_y_train = preprocessing(dataset_MLP,
                                                 scaling=True,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="frekuensi")
scaled_x_train

array([[0.47398844, 0.03333333],
       [0.20231214, 0.35      ],
       [0.57996146, 0.1       ],
       [0.10019268, 0.        ],
       [0.22543353, 0.55      ],
       [0.18304432, 0.05      ],
       [0.91907514, 0.45      ],
       [0.73025048, 0.26666667],
       [0.56069364, 0.1       ],
       [0.92870906, 0.        ],
       [0.94026975, 0.13333333],
       [0.76300578, 0.31666667],
       [0.01734104, 0.01666667],
       [0.01926782, 0.        ],
       [0.60693642, 0.55      ],
       [0.75722543, 0.15      ],
       [0.04431599, 0.13333333],
       [0.42967245, 0.16666667],
       [0.4566474 , 0.06666667],
       [0.69364162, 0.15      ],
       [0.96917148, 0.15      ],
       [0.83044316, 0.75      ],
       [0.19075145, 0.41666667],
       [0.08092486, 0.38333333],
       [0.14450867, 0.45      ],
       [0.50096339, 0.05      ],
       [0.24855491, 0.7       ],
       [0.46050096, 0.06666667],
       [0.71483622, 0.33333333],
       [0.15992293, 0.51666667],
       [0.

In [136]:
x_train, y_train = preprocessing(dataset_DT,
                                                 scaling=False,
                                                 categorical_columns=[],
                                                  unused_columns=[],
                                                 label="Aktif")
# scale frekuensi menjadi 0 sampai 1 biar mempermudah model DTnya
# scaler_freq = MinMaxScaler()
# frequency = scaler_freq.fit_transform(np.array([[x[1] for x in x_train]]).reshape(-1,1))
# for i,x in enumerate(x_train):
#     x_train[i][1] = frequency[i]

In [137]:
from sklearn import tree
model = tree.DecisionTreeRegressor()
model.fit(x_train, y_train)

DecisionTreeRegressor()

In [138]:
tranfusion_df = pd.read_csv("transfusion.csv")
tranfusion_df = tranfusion_df.drop(labels=["Monetary (c.c. blood)","Recency (months)","whether he/she donated blood in March 2007"],axis=1)
tranfusion_df = tranfusion_df[["Time (months)","Frequency (times)"]]

for i, row in tranfusion_df.iterrows():
    tranfusion_df.at[i,"Time (months)"] = row["Time (months)"]*4
print(tranfusion_df.head())

tranfusion_df.rename(columns = {'Time (weeks)':'Time (weeks)'}, inplace = True)
tranfusion_arr = tranfusion_df.to_numpy()

# tranfusion_arr =np.array(list(map(toWeek,tranfusion_arr)))
print(tranfusion_arr)
scaled_tranfusion_arr = scalerX.transform(tranfusion_arr)

   Time (months)  Frequency (times)
0            392                 50
1            112                 13
2            140                 16
3            180                 20
4            308                 24
[[392  50]
 [112  13]
 [140  16]
 ...
 [248   3]
 [156   1]
 [288   1]]


In [163]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(activation="relu",solver="adam",hidden_layer_sizes=(100,))
mlp.fit(scaled_x_train, scaled_y_train.ravel())
result = mlp.predict(scaled_x_train)
value=str(result[3])
value2=value.replace(',', '.')
float(value2)

C:\Users\c1419\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.35398431917328693

In [164]:
scaled_y_train
value=str(scaled_y_train[3][0])
value2=value.replace(',', '.')
float(value2)

0.018867924528301886

In [141]:
scaled_x_train

array([[0.47398844, 0.03333333],
       [0.20231214, 0.35      ],
       [0.57996146, 0.1       ],
       [0.10019268, 0.        ],
       [0.22543353, 0.55      ],
       [0.18304432, 0.05      ],
       [0.91907514, 0.45      ],
       [0.73025048, 0.26666667],
       [0.56069364, 0.1       ],
       [0.92870906, 0.        ],
       [0.94026975, 0.13333333],
       [0.76300578, 0.31666667],
       [0.01734104, 0.01666667],
       [0.01926782, 0.        ],
       [0.60693642, 0.55      ],
       [0.75722543, 0.15      ],
       [0.04431599, 0.13333333],
       [0.42967245, 0.16666667],
       [0.4566474 , 0.06666667],
       [0.69364162, 0.15      ],
       [0.96917148, 0.15      ],
       [0.83044316, 0.75      ],
       [0.19075145, 0.41666667],
       [0.08092486, 0.38333333],
       [0.14450867, 0.45      ],
       [0.50096339, 0.05      ],
       [0.24855491, 0.7       ],
       [0.46050096, 0.06666667],
       [0.71483622, 0.33333333],
       [0.15992293, 0.51666667],
       [0.

expectValue digunakan untuk menvalidasi output dari model MLP dan Decision Tree  
Berdasarkan https://www.halodoc.com/artikel/berapa-kali-boleh-donor-darah-dalam-setahun dikatakan aktif(pria) bila interval waktu donor 8-12 minggu

In [142]:
expectValue = list(map(lambda x: 1.0 if x[0]/x[1] <= 12.0 else 0.0,tranfusion_arr))

 **source dataset  : https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download**

In [143]:
MLPresult = mlp.predict(scaled_tranfusion_arr)
# DTresult = model.predict(tranfusion_arr) 

tranfusion_df["Predicted Value (MLP)"] = MLPresult
# tranfusion_df["Predicted Value (DT)"] = DTresult
tranfusion_df["Expected Value"] = expectValue

In [144]:
from IPython.display import display, HTML
# tranfusion_df
display(HTML("<h2>Hasil Perbandingan Deep Learning (MLP) dan Decision Tree (DT)</h2><p>source dataset: <a href='https://www.kaggle.com/datasets/ninalabiba/blood-transfusion-dataset?resource=download'>kaggle</a></p>"))
display(HTML(f"<div style='overflow-y:auto;max-height:200px'>{tranfusion_df.to_html()}</div>"))

,Time (months),Frequency (times),Predicted Value (MLP),Expected Value
0,392,50,0.141275,1.0
1,112,13,0.041608,1.0
2,140,16,0.079832,1.0
3,180,20,0.089149,1.0
4,308,24,0.097887,0.0
5,16,4,0.671507,1.0
6,56,7,0.047854,1.0
7,140,12,0.078582,1.0
8,88,9,0.022201,1.0
9,392,46,0.130897,1.0


In [145]:
# fig = plt.figure(figsize=(25,20))
# _ = tree.plot_tree(model, 
#                    feature_names=["Donor 1 tahun terakhir","Jumlah hari terakhir donor", "frekuensi donor per tahun"],  
#                    class_names=["aktif"],
#                    filled=True)

import graphviz
# DOT data
dot_data = tree.export_graphviz(model, out_file=None, 
                                feature_names=["Donor 1 tahun terakhir", "frekuensi donor per tahun"],  
                                class_names=["aktif"],
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph.render("decision_tree_graphivz")

'decision_tree_graphivz.png'

In [146]:
import pickle
pickle.dump(model, open("model", 'wb'))